# Segmenting and Clustering Neighborhoods in Toronto

## Case Study 1: Comparing number of vevues in Toronto and NewYork City to find similarity and dissimilarity.

## Case Study 2: Business Man want to open Coffee Shop in Toronto. Already Toronto city divided into 5 clusters. What is the best place to open coffee Shop. Peferred where less coffeeshops are present. 

# Initialization

In [1]:
!pip install bs4
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd #this module helps us to work of data frames

print("Install & Import Completed")

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=ebba2233fc7a9ce9877e0ee60254564dc881e4f7a316b5e62f01b603a4fac6f0
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
Install & Import Completed


# Using BeautifulSoup function to retrieve Toronto City neighborhoods from Wikipedia

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #URL
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib") 

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned': #Ignore cells with a borough that is Not assigned
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell) #combined into one row with the neighborhoods separated with a comma

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


# Reading CSV provide in case study where Latitude and Longitute is stored against Postal Code

In [3]:
GeoSpatial_Dataset_URL ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
df2 = pd.read_csv(GeoSpatial_Dataset_URL)
print(df2)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


# Merging dataframe & csv

In [4]:
newdf = pd.merge(df,df2,left_on='PostalCode',right_on='Postal Code')
newdf = newdf.drop('Postal Code', axis = 1)
newdf.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


    Additional Import

# Import Folium and other libraries

In [5]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

! pip install folium==0.5.0
import folium # plotting library
from folium.plugins import FastMarkerCluster
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Folium installed')
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 79 kB 10.8 MB/s eta 0:00:01
  Created wheel for folium: file

# Search Toronto City using Foursquare

In [6]:
CLIENT_ID = 'YCMYU5UCNLL10YKNWQHG2FAQLEKJA23C4RBQ3N5XCI1BVSUP' # your Foursquare ID
CLIENT_SECRET = '1V01DSEUGBHWEDTBUMPXECK34IVU3SZHHJ3FPZW54EIVPZKG' # your Foursquare Secret
ACCESS_TOKEN = 'WHSTD330S31P54LPJOC1HIFDBN5DYGAI3AHPQZKQPXBZC2LO' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
tlongitude = longitude
tlatitude = latitude


# Map Plotting of Toronto City 


In [7]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(newdf['Latitude'],newdf['Longitude'],newdf['Borough'],newdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

# Apply K Mean for cluster size 5

In [8]:
k=5
toronto_clustering = newdf.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
newdf.insert(0, 'Cluster Labels', kmeans.labels_)

# Plot Clusters on Toronto City

In [9]:


#create cluster plot

# create map
map_clusters = folium.Map(location=[latitude,longitude],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(newdf['Latitude'], newdf['Longitude'], newdf['Neighborhood'], newdf['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Finding top 100 places search for Newyork and Toronto using Foursquare

In [10]:
import requests
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, tlatitude, tlongitude, VERSION, radius, LIMIT)

tresults = requests.get(url).json()
#print('There are top {} around Toronto.'.format(len(tresults['response']['groups'][0]['items'])))

latitude = 40.715337
longitude = -74.008848
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
nresults = requests.get(url).json()
print('There are top {} around New York City.'.format(len(nresults['response']['groups'][0]['items'])))
titems = tresults['response']['groups'][0]['items']
tdataframe = json_normalize(titems)
nitems = nresults['response']['groups'][0]['items']
ndataframe = json_normalize(nitems)


There are top 100 around New York City.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


# Define functions for filtering

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
def get_top10_venue(dframe): 
    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dframe.columns if col.startswith('venue.location.')] + ['venue.id']
    dataframe_filtered = dframe.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean columns
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

    return dataframe_filtered.head(10)['categories']

# Fliter Applied on NewYork City Data

In [12]:

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in ndataframe.columns if col.startswith('venue.location.')] + ['venue.id']
ndataframe_filtered = ndataframe.loc[:, filtered_columns]

# filter the category for each row
ndataframe_filtered['venue.categories'] = ndataframe_filtered.apply(get_category_type, axis=1)

# clean columns
ndataframe_filtered.columns = [col.split('.')[-1] for col in ndataframe_filtered.columns]
#ndataframe

# Filter Applied on Toronta City Data

In [13]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in tdataframe.columns if col.startswith('venue.location.')] + ['venue.id']
tdataframe_filtered = tdataframe.loc[:, filtered_columns]

# filter the category for each row
tdataframe_filtered['venue.categories'] = tdataframe_filtered.apply(get_category_type, axis=1)

# clean columns
tdataframe_filtered.columns = [col.split('.')[-1] for col in tdataframe_filtered.columns]
#tdataframe_filtered.head(6)

# Top 10 Categories places in Toronto

In [14]:
tdataframe_filtered["categories"].value_counts()[0:10]

Clothing Store             7
Coffee Shop                6
American Restaurant        2
Diner                      2
Bookstore                  2
Plaza                      2
Theater                    2
New American Restaurant    2
Seafood Restaurant         2
Sushi Restaurant           2
Name: categories, dtype: int64

# Top 10 Categories places in NewYork

In [15]:
ndataframe_filtered["categories"].value_counts()[0:10]

Coffee Shop             5
Italian Restaurant      4
Indian Restaurant       3
Gym                     3
French Restaurant       3
Gym / Fitness Center    3
Park                    3
Spa                     3
American Restaurant     3
Falafel Restaurant      2
Name: categories, dtype: int64

# By looking into the above stat we found:

### Two cities similarity: Coffee Shops are liked by people of both cities.
### Two cities dissimilarity: Italian Restaurant are liked by people of NewYork whereas Toronto people like American Restaurant.

# City of Choice - Toronto --> Opening a Coffee Shop

The cluster where less no of coffee shop present, system will be predicting

# Defining cluster classes to store data

In [16]:
from collections import Counter

class Clust:
    
    def __init__(self,id1):
        self.id = id1
        self.vlist = []
    def addv(self,ldata):
        self.vlist = self.vlist + ldata
    def cvenue(self):
        print(Counter(self.vlist))
    def pvenue(self):
        print(self.vlist)
    def ccvenue(self,s):
        self.dic = Counter(self.vlist)
        return self.dic[s]

ClustList = []
for i in range(5):
    ClustList.append(Clust(i))


# Explore cluster of Toronto City - Count Shop Categories for each Cluster

In [17]:
for lat, lon, cluster in zip(newdf['Latitude'], newdf['Longitude'], newdf['Cluster Labels']):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lon, VERSION, radius, LIMIT)
    #url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, newdf['Latitude'], newdf['Longitude'], VERSION, radius, LIMIT)
    xresults = requests.get(url).json()

    xitems = xresults['response']['groups'][0]['items']
    xdataframe = json_normalize(xitems)
    if len(xdataframe) > 0:
        ClustList[cluster].addv(get_top10_venue(xdataframe).tolist())


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


# Predicting Highest and Lowest Cluster ID based on Coffee Shop Count

In [18]:
hv = lv = 0
hc = lc = 0

for i in range(5):
    if i == 0:
        hv = ClustList[i].ccvenue('Coffee Shop')  
    elif hv < ClustList[i].ccvenue('Coffee Shop'):
        hv = ClustList[i].ccvenue('Coffee Shop')
        hc = i

for i in range(5):
    if i == 0:
        lv = ClustList[i].ccvenue('Coffee Shop')
    elif lv > ClustList[i].ccvenue('Coffee Shop'):
        lv = ClustList[i].ccvenue('Coffee Shop')
        lc = i
      
for x in range(5):
    print("Cluster",str(x),str(),ClustList[x].ccvenue('Coffee Shop'))

Cluster 0  8
Cluster 1  9
Cluster 2  23
Cluster 3  2
Cluster 4  5


In [23]:
print("Will Open Coffee Shop is Cluster:", str(lc), ", Currently Coffee Shop Count in this cluster is:" + str(lv) )

Will Open Coffee Shop is Cluster: 3 , Currently Coffee Shop Count in this cluster is:2


# Plotting Recommended Cluster on Map

In [24]:
#create cluster plot

# create map
map_clusters = folium.Map(location=[tlatitude,tlongitude],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(newdf['Latitude'], newdf['Longitude'], newdf['Neighborhood'], newdf['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    if cluster == lc:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup="Proposed Coffee Shop",
            color='blue',
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
    if cluster == hc:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup="Existing Coffee Shop",
            color='red',
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
map_clusters

# In Red : Cluster in Toronto with highest no of coffee shops
# In Blue : Cluster in Toronto with less no of coffee shops